# Day 14

## Part One

<p>The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has <a href="https://en.wikipedia.org/wiki/Polymerization" target="_blank">polymerization</a> equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.</p>
<p>The submarine manual contains <span title="HO&#xa;&#xa;HO -&gt; OH">instructions</span> for finding the optimal polymer formula; specifically, it offers a <em>polymer template</em> and a list of <em>pair insertion</em> rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.</p>
<p>For example:</p>
<pre><code>NNCB

CH -&gt; B
HH -&gt; N
CB -&gt; H
NH -&gt; C
HB -&gt; C
HC -&gt; B
HN -&gt; C
NN -&gt; C
BH -&gt; H
NC -&gt; B
NB -&gt; B
BN -&gt; B
BB -&gt; N
BC -&gt; B
CC -&gt; N
CN -&gt; C
</code></pre>
<p>The first line is the <em>polymer template</em> - this is the starting point of the process.</p>
<p>The following section defines the <em>pair insertion</em> rules. A rule like <code>AB -&gt; C</code> means that when elements <code>A</code> and <code>B</code> are immediately adjacent, element <code>C</code> should be inserted between them. These insertions all happen simultaneously.</p>
<p>So, starting with the polymer template <code>NNCB</code>, the first step simultaneously considers all three pairs:</p>
<ul>
<li>The first pair (<code>NN</code>) matches the rule <code>NN -&gt; C</code>, so element <code><em>C</em></code> is inserted between the first <code>N</code> and the second <code>N</code>.</li>
<li>The second pair (<code>NC</code>) matches the rule <code>NC -&gt; B</code>, so element <code><em>B</em></code> is inserted between the <code>N</code> and the <code>C</code>.</li>
<li>The third pair (<code>CB</code>) matches the rule <code>CB -&gt; H</code>, so element <code><em>H</em></code> is inserted between the <code>C</code> and the <code>B</code>.</li>
</ul>
<p>Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.</p>
<p>After the first step of this process, the polymer becomes <code>N<em>C</em>N<em>B</em>C<em>H</em>B</code>.</p>
<p>Here are the results of a few steps using the above rules:</p>
<pre><code>Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
</code></pre>
<p>This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, <code>B</code> occurs 1749 times, <code>C</code> occurs 298 times, <code>H</code> occurs 161 times, and <code>N</code> occurs 865 times; taking the quantity of the most common element (<code>B</code>, 1749) and subtracting the quantity of the least common element (<code>H</code>, 161) produces <code>1749 - 161 = <em>1588</em></code>.</p>
<p>Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. <em>What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?</em></p>

---

In [4]:
# Initialise
with open('Day14.in') as f:
    template, rules = f.read().split('\n\n')
    rules = [tuple(r.split(' -> ')) for r in rules[:-1].split('\n')]

class Template(str):
    """
    This was useful for testing, but it's not the right data structure
    to solve part one or two quickly. I thought the functions were 
    pretty cool though so I've left them in
    """
    def findall(self, substring):
        idxs = []
        s = self
        i = 0
        j = 0
        n = len(substring)
        while True:
            j = s.find(substring)
            if j > -1:
                idxs.append(i+j)
                i += j + 1
                s = s[j+1:]
            else:
                break
        return idxs
    
    def insertion(self, pattern, char):
        idx = self.findall(pattern)
        c = self
        s = ''
        n = len(pattern)
        j = 0
        for i in idx:
            s += self[j:i+1] + char + self[i+1]
            j = i + n
        s += self[j:]
        return Template(s)

template = Template(template)

In [5]:
def generate_pairs_dict(t=template):
    """
    Take a template t and turn it into a dictionary of
    pairs and their count in the string. A character is
    represented in two separate pairs if it is not at 
    either end.
    
    NOTE: This deliberately leaves an entry for the final char
          in t so that it is counted.
    
    e.g. "ABBBC" -> {"AB": 1, "BB": 2, "BC":1, "C":1}
    """
    pairs = {}

    for i in range(len(t)):
        p = Template(t[i:i+2])
        if p in pairs:
            pairs[p] += 1
        else:
            pairs[p] = 1
    return pairs

def count_chars(pairs):
    """
    Takes a dictionary of pairs and returns the number of times
    each character would appear in the theoretical string.
    """
    # Initialise list of chars and appearance counting function
    chars = set([pair[0] for pair in pairs])
    appear = lambda pairs, char: sum([pairs[pair] for pair in pairs if char == pair[0]])
    
    return {char: appear(pairs, char) for char in chars}

def insertion(pairs, rules):
    """
    Takes a template (in the form of a dictionary of pairs) and rules and simulates one
    round of insertions on pairs based on rules.
    """
    # Delta will track all the new additions and deletions
    # Need to make sure we don't alter pairs as we generate insertions
    delta = {}
    for pair, insert in rules:        
        if pair in pairs:
            n = pairs[pair]
            if ((left:=(pair[0]+insert)) in delta):
                delta[left] += n
            else:
                delta[left] = n
            
            if ((right:=(insert+pair[1])) in delta):
                delta[right] += n
            else:
                delta[right] = n
            
            if pair in delta:
                delta[pair] -= n
            else:
                delta[pair] = -n
        
    return {
        p: d for p in ukeys(pairs, delta) if (d:= add_d(pairs, delta, p)) != 0
    }

# Helper functions for dictionary arithmetic
add_d = lambda d1, d2, pair: (d1[pair] if pair in d1 else 0) + (d2[pair] if pair in d2 else 0)
ukeys = lambda d1, d2: set(d1.keys()) | set(d2.keys())

# Initialise pairs dict
pairs = generate_pairs_dict()

# Perform 10 iterations 
# ( omg I thought it said 100 when I originally read it which led me down this path, )
# ( I think my Template class might have actually worked -_-                         )
for _ in range(10):
    pairs = insertion(pairs, rules)    
    
# Solution
counts = count_chars(pairs)
print("Solution:", max(counts.values()) - min(counts.values()))

Solution: 3009


---

## Part Two

<p>The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of <em>40 steps</em> should do it.</p>
<p>In the above example, the most common element is <code>B</code> (occurring <code>2192039569602</code> times) and the least common element is <code>H</code> (occurring <code>3849876073</code> times); subtracting these produces <code><em>2188189693529</em></code>.</p>
<p>Apply <em>40</em> steps of pair insertion to the polymer template and find the most and least common elements in the result. <em>What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?</em></p>

---

In [6]:
# Perform 30 more iterations 
for _ in range(30):
    pairs = insertion(pairs, rules)    
    
# Solution
counts = count_chars(pairs)
print("Solution:", max(counts.values()) - min(counts.values()))

Solution: 3459822539451


---